### Cardiac Arrhythmia Multy-Class Classification 

Analyze data and address missing data if there is any. 

Decide aboute a good evaluation strategy and justify your choice. 

Find the best parameters for the following classification models: 
- KNN classifcation 
- Logistic Regression
- Linear Supprt Vector Machine
- Kerenilzed Support Vector Machine
- Decision Tree
- Random Forest 

Then use different bagging and boosting methods to boost the results? Do you see any significant change? Why or why not? 

Next, use data reduction method you have learned in class to reduce the size of data, and agian try above models. Do you get better results? Justify your answer. 

<font color = 'red'>Due date for full credit: April 4, 11:59 PM
    <br>
    Due date for partial credit: April 6, 11:59 PM.
    <br> No submission will be accepted after April 6. 
    <br> Please note that your term paper is also due April 6. 
    </font>

# 1. Data Reading
Starting with reading the data file and creating a dataframe for putting the output of all the models that 
 we will be running. The purpose of doing this is it will be easy for us to compare the models.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn import preprocessing
from matplotlib import pyplot as plt

data = pd.read_csv('cardiac_arrhythmia1.csv')
output = pd.DataFrame(index=None, columns=['model','train_Rsquare', 'test_Rsquare', 'train_MSE','test_MSE'])
data.describe()

,Age,Sex,Height,Weight,QRS_duration,P-R_interval,Q-T_interval,T_interval,P_interval,QRS,...,V6_Amp_Q,V6_Amp_R,V6_Amp_S,V6_Amp_R',V6_Amp_S',V6_Amp_P,V6_Amp_T,V6_QRSA,V6_QRSTA,cardiac_arrhythmia
count,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,...,452.000000,452.000000,452.000000,452.000000,452.0,452.000000,452.000000,452.000000,452.000000,452.000000
mean,46.471239,0.550885,166.188053,68.170354,88.920354,155.152655,367.207965,169.949115,90.004425,33.676991,...,-0.278982,9.048009,-1.457301,0.003982,0.0,0.514823,1.222345,19.326106,29.473230,3.880531
std,16.466631,0.497955,37.170340,16.590803,15.364394,44.842283,33.385421,35.633072,25.826643,45.431434,...,0.548876,3.472862,2.002430,0.050118,0.0,0.347531,1.426052,13.503922,18.493927,4.407097
min,0.000000,0.000000,105.000000,6.000000,55.000000,0.000000,232.000000,108.000000,0.000000,-172.000000,...,-4.100000,0.000000,-28.600000,0.000000,0.0,-0.800000,-6.000000,-44.200000,-38.600000,1.000000
25%,36.000000,0.000000,160.000000,59.000000,80.000000,142.000000,350.000000,148.000000,79.000000,3.750000,...,-0.425000,6.600000,-2.100000,0.000000,0.0,0.400000,0.500000,11.450000,17.550000,1.000000
50%,47.000000,1.000000,164.000000,68.000000,86.000000,157.000000,367.000000,162.000000,91.000000,40.000000,...,0.000000,8.800000,-1.100000,0.000000,0.0,0.500000,1.350000,18.100000,27.900000,1.000000
75%,58.000000,1.000000,170.000000,79.000000,94.000000,175.000000,384.000000,179.000000,102.000000,66.000000,...,0.000000,11.200000,0.000000,0.000000,0.0,0.700000,2.100000,25.825000,41.125000,6.000000
max,83.000000,1.000000,780.000000,176.000000,188.000000,524.000000,509.000000,381.000000,205.000000,169.000000,...,0.000000,23.600000,0.000000,0.800000,0.0,2.400000,6.000000,88.800000,115.900000,16.000000


# Handling the missing value
While going through the dataset we observed that out of 279 columns 5 columns have missing value in the form of '?'.
The approach which we will following is, first replacing '?' with numpy NAN and then imputing the mean.

In [4]:
import numpy as np
data['J'] = data['J'].replace('?',np.NaN)
data['Heart_Rate'] = data['Heart_Rate'].replace('?',np.NaN)
data['P'] = data['P'].replace('?',np.NaN)
data['T'] = data['T'].replace('?',np.NaN)
data['QRST'] = data['QRST'].replace('?',np.NaN)

# Spliting the dataset
Segregating the whole dataset into X and Y 

In [5]:
Data_Y = data.cardiac_arrhythmia.values.ravel()
Data_X=data.drop('cardiac_arrhythmia', 1)

In [6]:
np.unique(Data_Y, return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 14, 15, 16], dtype=int64),
 array([245,  44,  15,  15,  13,  25,   3,   2,   9,  50,   4,   5,  22],
       dtype=int64))

# Drop column
We can observe that column J have a lot of missing value. It will not be a good practice to impute mean values in this column.
Better option will be that we drop this column

In [7]:
Data_X.drop(columns=['J'])

,Age,Sex,Height,Weight,QRS_duration,P-R_interval,Q-T_interval,T_interval,P_interval,QRS,...,V6_Amp_JJ,V6_Amp_Q,V6_Amp_R,V6_Amp_S,V6_Amp_R',V6_Amp_S',V6_Amp_P,V6_Amp_T,V6_QRSA,V6_QRSTA
0,75,0,190,80,91,193,371,174,121,-16,...,-0.3,0.0,9.0,-0.9,0.0,0,0.9,2.9,23.3,49.4
1,56,1,165,64,81,174,401,149,39,25,...,-0.5,0.0,8.5,0.0,0.0,0,0.2,2.1,20.4,38.8
2,54,0,172,95,138,163,386,185,102,96,...,0.9,0.0,9.5,-2.4,0.0,0,0.3,3.4,12.3,49.0
3,55,0,175,94,100,202,380,179,143,28,...,0.1,0.0,12.2,-2.2,0.0,0,0.4,2.6,34.6,61.6
4,75,0,190,80,88,181,360,177,103,-16,...,-0.4,0.0,13.1,-3.6,0.0,0,-0.1,3.9,25.4,62.8
5,13,0,169,51,100,167,321,174,91,107,...,0.0,-0.6,12.2,-2.8,0.0,0,0.9,2.2,13.5,31.1
6,40,1,160,52,77,129,377,133,77,77,...,-0.4,0.0,6.5,0.0,0.0,0,0.4,1.0,14.3,20.5
7,49,1,162,54,78,0,376,157,70,67,...,-0.3,0.0,8.2,-1.9,0.0,0,0.1,0.5,15.8,19.8
8,44,0,168,56,84,118,354,160,63,61,...,0.1,0.0,7.0,-1.3,0.0,0,0.6,2.1,12.5,30.9
9,50,1,167,67,89,130,383,156,73,85,...,-0.5,-0.6,10.8,-1.7,0.0,0,0.8,0.9,20.1,25.1


# Handling missing value
We are imputing mean in place of missing values

In [8]:
from sklearn.preprocessing import Imputer
z=Imputer(missing_values=np.nan, strategy='mean', axis=1).fit_transform(Data_X)
Data_X = pd.DataFrame(data=z,columns=Data_X.columns.values)
Data_X.isnull().sum()

Age                 0
Sex                 0
Height              0
Weight              0
QRS_duration        0
P-R_interval        0
Q-T_interval        0
T_interval          0
P_interval          0
QRS                 0
T                   0
P                   0
QRST                0
J                   0
Heart_Rate          0
DI_Q_wave           0
DI_R_wave           0
DI_S_wave           0
DI_R'_wave          0
DI_S'_wave          0
DI_Intrinsic_def    0
DI_Ragged_R         0
DI_DD_R             0
DI_Ragged_P         0
DI_DD_P             0
DI_Ragged_T         0
DI_DD_T             0
DII_Q_wave          0
DII_R_wave          0
DII_S_wave          0
                   ..
V4_Amp_JJ           0
V4_Amp_Q            0
V4_Amp_R            0
V4_Amp_S            0
V4_Amp_R'           0
V4_Amp_S'           0
V4_Amp_P            0
V4_Amp_T            0
V4_QRSA             0
V4_QRSTA            0
V5_Amp_JJ           0
V5_Amp_Q            0
V5_Amp_R            0
V5_Amp_S            0
V5_Amp_R' 

#  Good evaluation strategy
As the dependent variabe is a categorical variable we will be using classification models. The best evaluation strategy for classification models is comparing the precision and recall. We know for a fact that R-sqaured and MSE scores are used extensively for checking the accuracy of a regression model where independent variable is a continous. However when we run classification models precision and recall are the best estimators of accuracy. Our main aim is to reduce the recall by improving the model.

# Spliting down into both train and test data set

In [9]:
data_train_x, data_test_x, data_train_y, data_test_y = train_test_split(Data_X, Data_Y
                                                                        , random_state=2)
# Scaling the data (MIN MAX Scaling)
print('Shape of train {}, shape of test {}'.format(data_train_x.shape, data_test_x.shape))

Shape of train (339, 279), shape of test (113, 279)


# Scaling
As the variables are on different scale it will be helpful if we bring them all on the same scale. Scaling improves the performance of the models

In [10]:
from sklearn.preprocessing import MinMaxScaler

#MinMax
MinMax = MinMaxScaler(feature_range= (0,1))
data_train_x = MinMax.fit_transform(data_train_x)
data_test_x = MinMax.transform(data_test_x)

# 2. Modeling
After taking care of the data we will be starting with the model creation. 

# KNN Regression

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

## We are creating a grid for which all n_neighbors values are to be used for cross validation

param_grid={'weights':['distance', 'uniform'], 'n_neighbors':range(1,100)}

## Using Grid search for exhaustive searching

grid_search = GridSearchCV( KNeighborsClassifier(),param_grid, cv = 10)
grid_search.fit(data_train_x, data_train_y)


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=10, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'weights': ['distance', 'uniform'], 'n_neighbors': range(1, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [12]:
from sklearn.metrics import r2_score, mean_squared_error
train_Rsquare = grid_search.score(data_train_x, data_train_y)
test_Rsquare = grid_search.score(data_test_x, data_test_y)
train_MSE = mean_squared_error(data_train_y, grid_search.predict(data_train_x))
test_MSE = mean_squared_error(data_test_y, grid_search.predict(data_test_x))
output = output.append(pd.Series({'model':'KNN Classifier','train_Rsquare':train_Rsquare, 'test_Rsquare':test_Rsquare, 'train_MSE':train_MSE,'test_MSE':test_MSE}),ignore_index=True )
output

,model,train_Rsquare,test_Rsquare,train_MSE,test_MSE
0,KNN Classifier,1.0,0.504425,0.0,25.973451


# R-squared, MSE vs precision, Recall
We know for a fact that R-sqaured and MSE scores are used extensively for checking the accuracy of a regression model where independent variable is a continous. However when we run classification models precision and recall are the best estimators of accuracy

In [13]:
pd.DataFrame(grid_search.cv_results_)
print(grid_search.best_estimator_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='distance')


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\

In [14]:
from sklearn.metrics import classification_report
knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=6, p=2,
           weights='distance')
knn.fit(data_train_x, data_train_y)
pred = knn.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.51      0.96      0.67        57
          2       0.00      0.00      0.00        14
          3       0.00      0.00      0.00         6
          4       1.00      0.25      0.40         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       1.00      0.11      0.20         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.37      0.50      0.37       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Output Evaluation
In the whole process of model creation, our aim will be to achieve maximum Precision (no false positive) and maximum Recall (no false negative) there needs to be an absence of type I and II errors

# Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }

grid_search_log = GridSearchCV(LogisticRegression(penalty='l2'), param_grid, cv=5)
grid_search_log.fit(data_train_x, data_train_y)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [16]:
from sklearn.metrics import r2_score, mean_squared_error
train_Rsquare = grid_search_log.score(data_train_x, data_train_y)
test_Rsquare = grid_search_log.score(data_test_x, data_test_y)
train_MSE = mean_squared_error(data_train_y, grid_search_log.predict(data_train_x))
test_MSE = mean_squared_error(data_test_y, grid_search_log.predict(data_test_x))
output = output.append(pd.Series({'model':'Logistic Regression','train_Rsquare':train_Rsquare, 'test_Rsquare':test_Rsquare, 'train_MSE':train_MSE,'test_MSE':test_MSE}),ignore_index=True )
output

,model,train_Rsquare,test_Rsquare,train_MSE,test_MSE
0,KNN Classifier,1.000000,0.504425,0.000000,25.973451
1,Logistic Regression,0.823009,0.699115,9.351032,17.991150


In [17]:
pd.DataFrame(grid_search_log.cv_results_)
print(grid_search_log.best_estimator_)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\

In [18]:
log = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
log.fit(data_train_x, data_train_y)
pred = log.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.69      0.96      0.80        57
          2       0.55      0.43      0.48        14
          3       1.00      1.00      1.00         6
          4       0.75      0.75      0.75         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       1.00      0.25      0.40         4
         10       0.88      0.78      0.82         9
         15       0.00      0.00      0.00         2
         16       1.00      0.17      0.29         6

avg / total       0.65      0.70      0.64       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Linear Supprt Vector Machine

In [19]:
from sklearn.svm import LinearSVC

param_grid = {'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter':[1000,10000] }

grid_search_SVC = GridSearchCV(LinearSVC(random_state=0), param_grid, cv=5)
grid_search_SVC.fit(data_train_x, data_train_y)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'max_iter': [1000, 10000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
train_Rsquare = grid_search_SVC.score(data_train_x, data_train_y)
test_Rsquare = grid_search_SVC.score(data_test_x, data_test_y)
train_MSE = mean_squared_error(data_train_y, grid_search_SVC.predict(data_train_x))
test_MSE = mean_squared_error(data_test_y, grid_search_SVC.predict(data_test_x))
output = output.append(pd.Series({'model':'Linear SVC','train_Rsquare':train_Rsquare, 'test_Rsquare':test_Rsquare, 'train_MSE':train_MSE,'test_MSE':test_MSE}),ignore_index=True )
output

,model,train_Rsquare,test_Rsquare,train_MSE,test_MSE
0,KNN Classifier,1.000000,0.504425,0.000000,25.973451
1,Logistic Regression,0.823009,0.699115,9.351032,17.991150
2,Linear SVC,0.828909,0.699115,10.351032,17.548673


In [21]:
pd.DataFrame(grid_search_SVC.cv_results_)
print(grid_search_SVC.best_estimator_)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\

In [22]:
linearsvc = LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)
linearsvc.fit(data_train_x, data_train_y)
pred = linearsvc.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.68      0.95      0.79        57
          2       0.60      0.43      0.50        14
          3       0.86      1.00      0.92         6
          4       1.00      0.75      0.86         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.67      0.50      0.57         4
         10       0.78      0.78      0.78         9
         15       0.00      0.00      0.00         2
         16       1.00      0.17      0.29         6

avg / total       0.64      0.70      0.64       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Kerenilzed Support Vector Machine

In [23]:
from sklearn.svm import SVC

param_grid = {'C':[0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'gamma':[0.001, 0.01, 0.1, 0.5, 1, 10]}

grid_search_KSVC = GridSearchCV(SVC(kernel = 'rbf'), param_grid, cv=5)
grid_search_KSVC.fit(data_train_x, data_train_y)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'gamma': [0.001, 0.01, 0.1, 0.5, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [24]:
train_Rsquare = grid_search_KSVC.score(data_train_x, data_train_y)
test_Rsquare = grid_search_KSVC.score(data_test_x, data_test_y)
train_MSE = mean_squared_error(data_train_y, grid_search_KSVC.predict(data_train_x))
test_MSE = mean_squared_error(data_test_y, grid_search_KSVC.predict(data_test_x))
output = output.append(pd.Series({'model':'Kernel SVC','train_Rsquare':train_Rsquare, 'test_Rsquare':test_Rsquare, 'train_MSE':train_MSE,'test_MSE':test_MSE}),ignore_index=True )
output

,model,train_Rsquare,test_Rsquare,train_MSE,test_MSE
0,KNN Classifier,1.000000,0.504425,0.000000,25.973451
1,Logistic Regression,0.823009,0.699115,9.351032,17.991150
2,Linear SVC,0.828909,0.699115,10.351032,17.548673
3,Kernel SVC,0.893805,0.707965,6.628319,23.070796


In [25]:
pd.DataFrame(grid_search_KSVC.cv_results_)
print(grid_search_KSVC.best_estimator_)

SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\

In [26]:
svc = SVC(C=50, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
svc.fit(data_train_x, data_train_y)
pred = svc.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.68      0.91      0.78        57
          2       0.80      0.57      0.67        14
          3       1.00      1.00      1.00         6
          4       0.80      1.00      0.89         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.75      0.75      0.75         4
         10       1.00      0.67      0.80         9
         15       1.00      0.50      0.67         2
         16       0.00      0.00      0.00         6

avg / total       0.65      0.71      0.66       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier

param_grid = {'max_features':[None,'auto', 'log2'], 'max_depth':[5,10,15,20,50]}

grid_search_DT = GridSearchCV(DecisionTreeClassifier(random_state = 10), param_grid, cv=5)
grid_search_DT.fit(data_train_x, data_train_y)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GridSearchCV(cv=5, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': [None, 'auto', 'log2'], 'max_depth': [5, 10, 15, 20, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
train_Rsquare = grid_search_DT.score(data_train_x, data_train_y)
test_Rsquare = grid_search_DT.score(data_test_x, data_test_y)
train_MSE = mean_squared_error(data_train_y, grid_search_DT.predict(data_train_x))
test_MSE = mean_squared_error(data_test_y, grid_search_DT.predict(data_test_x))
output = output.append(pd.Series({'model':'Decision Tree Classifier','train_Rsquare':train_Rsquare, 'test_Rsquare':test_Rsquare, 'train_MSE':train_MSE,'test_MSE':test_MSE}),ignore_index=True )
output

,model,train_Rsquare,test_Rsquare,train_MSE,test_MSE
0,KNN Classifier,1.000000,0.504425,0.000000,25.973451
1,Logistic Regression,0.823009,0.699115,9.351032,17.991150
2,Linear SVC,0.828909,0.699115,10.351032,17.548673
3,Kernel SVC,0.893805,0.707965,6.628319,23.070796
4,Decision Tree Classifier,0.958702,0.663717,4.126844,21.292035


In [29]:
pd.DataFrame(grid_search_DT.cv_results_)
print(grid_search_DT.best_estimator_)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best')


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\

In [30]:
dt = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=10,
            splitter='best')
dt.fit(data_train_x, data_train_y)
pred = dt.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.76      0.88      0.81        57
          2       0.78      0.50      0.61        14
          3       0.67      0.67      0.67         6
          4       0.25      0.25      0.25         4
          5       0.00      0.00      0.00         4
          6       0.62      0.83      0.71         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       0.50      0.78      0.61         9
         14       0.00      0.00      0.00         0
         15       0.00      0.00      0.00         2
         16       0.50      0.17      0.25         6

avg / total       0.62      0.66      0.63       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

#Tuning ridge on new dataset
param_grid = {"max_depth": [3, 5],
              "max_features": sp_randint(1, 40),
              "min_samples_split": sp_randint(2, 30),
              "min_samples_leaf": sp_randint(1, 20),
              "bootstrap": [True, False]}
grid_search_RF = RandomizedSearchCV(RandomForestClassifier(n_estimators=1000), param_distributions=param_grid,
                                   n_iter=30, random_state=0,n_jobs=-1)
grid_search_RF.fit(data_train_x, data_train_y)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=30, n_jobs=-1,
          param_distributions={'max_depth': [3, 5], 'max_features': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000111812B4C50>, 'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000111812B4D68>, 'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001118145C940>, 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=0, refit=True,
          return_tr

In [32]:
train_Rsquare = grid_search_RF.score(data_train_x, data_train_y)
test_Rsquare = grid_search_RF.score(data_test_x, data_test_y)
train_MSE = mean_squared_error(data_train_y, grid_search_RF.predict(data_train_x))
test_MSE = mean_squared_error(data_test_y, grid_search_RF.predict(data_test_x))
output = output.append(pd.Series({'model':'Random Forest Classifier','train_Rsquare':train_Rsquare, 'test_Rsquare':test_Rsquare, 'train_MSE':train_MSE,'test_MSE':test_MSE}),ignore_index=True )
output

,model,train_Rsquare,test_Rsquare,train_MSE,test_MSE
0,KNN Classifier,1.000000,0.504425,0.000000,25.973451
1,Logistic Regression,0.823009,0.699115,9.351032,17.991150
2,Linear SVC,0.828909,0.699115,10.351032,17.548673
3,Kernel SVC,0.893805,0.707965,6.628319,23.070796
4,Decision Tree Classifier,0.958702,0.663717,4.126844,21.292035
5,Random Forest Classifier,0.755162,0.637168,17.731563,22.663717


In [33]:
pd.DataFrame(grid_search_RF.cv_results_)
print(grid_search_RF.best_estimator_)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=5, max_features=36, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=25,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\utils\

In [34]:
rf = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=5, max_features=36, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=25,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
rf.fit(data_train_x, data_train_y)
pred = rf.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.64      0.98      0.78        57
          2       0.58      0.50      0.54        14
          3       1.00      0.50      0.67         6
          4       0.00      0.00      0.00         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       0.73      0.89      0.80         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.51      0.65      0.56       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# 3. Bagging
We will be using bagging to improve our earlier model with best parameters

##   KNN with bagging

In [35]:
from sklearn.ensemble import BaggingClassifier

KNN_bagging = BaggingClassifier(knn, n_estimators = 100, bootstrap = True)
KNN_bagging.fit(data_train_x,data_train_y)
pred = KNN_bagging.predict(data_test_x)
print(classification_report(data_test_y,pred))


             precision    recall  f1-score   support

          1       0.51      0.96      0.67        57
          2       0.00      0.00      0.00        14
          3       0.00      0.00      0.00         6
          4       1.00      0.25      0.40         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       1.00      0.11      0.20         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.37      0.50      0.37       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Logistic regression with bagging

In [36]:
log_bagging = BaggingClassifier(log , n_estimators = 100, max_features = 200 ,bootstrap = True, oob_score = True)
log_bagging.fit(data_train_x, data_train_y)
pred = log_bagging.predict(data_test_x)
print(classification_report(data_test_y,pred))


             precision    recall  f1-score   support

          1       0.62      0.98      0.76        57
          2       0.44      0.29      0.35        14
          3       1.00      0.83      0.91         6
          4       1.00      0.50      0.67         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       0.67      0.44      0.53         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.51      0.63      0.54       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Linear SVC with bagging

In [37]:
linearsvc_bagging = BaggingClassifier(linearsvc , n_estimators = 100, max_features = 200 ,
                                      bootstrap = True, oob_score = True)
linearsvc_bagging.fit(data_train_x, data_train_y)
pred = linearsvc_bagging.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.65      0.96      0.78        57
          2       0.57      0.29      0.38        14
          3       0.86      1.00      0.92         6
          4       1.00      0.75      0.86         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.67      0.50      0.57         4
         10       0.75      0.67      0.71         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.57      0.67      0.60       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Kernalized SVC with bagging

In [38]:
svc_bagging = BaggingClassifier(svc , n_estimators = 100, max_features = 200 ,
                                      bootstrap = True, oob_score = True)
svc_bagging.fit(data_train_x, data_train_y)
pred = svc_bagging.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.66      0.96      0.79        57
          2       0.70      0.50      0.58        14
          3       1.00      1.00      1.00         6
          4       0.80      1.00      0.89         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.75      0.75      0.75         4
         10       1.00      0.56      0.71         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.61      0.71      0.64       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Decision Tree with bagging

In [39]:
dt_bagging = BaggingClassifier(dt , n_estimators = 100, max_features = 200 ,
                                      bootstrap = True, oob_score = True)
dt_bagging.fit(data_train_x, data_train_y)
pred = dt_bagging.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.73      0.95      0.82        57
          2       0.62      0.57      0.59        14
          3       1.00      0.50      0.67         6
          4       1.00      0.25      0.40         4
          5       1.00      0.25      0.40         4
          6       0.83      0.83      0.83         6
          8       0.00      0.00      0.00         1
          9       1.00      0.75      0.86         4
         10       0.67      0.89      0.76         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.70      0.73      0.69       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Random forest with bagging

In [40]:
rf_bagging = BaggingClassifier(rf , n_estimators = 100, max_features = 200 ,
                                      bootstrap = True, oob_score = True)
rf_bagging.fit(data_train_x, data_train_y)
pred = rf_bagging.predict(data_test_x)
print(classification_report(data_test_y,pred))

             precision    recall  f1-score   support

          1       0.63      1.00      0.77        57
          2       0.60      0.43      0.50        14
          3       1.00      0.33      0.50         6
          4       0.00      0.00      0.00         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       0.80      0.89      0.84         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.51      0.65      0.54       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Boosting
The main idea behind using boosting is to convert weak learners 

# Ada boosting with Logistic regression

In [41]:
from sklearn.ensemble import AdaBoostClassifier
param_grid = {'learning_rate':[0.0001,0.001,0.01,0.1,1]}
adaboost_log = GridSearchCV(AdaBoostClassifier(base_estimator = log,random_state = 0), param_grid, cv=5,return_train_score=True)
adaboost_log.fit(data_train_x, data_train_y)

pred = adaboost_log.predict(data_test_x)
print(classification_report(data_test_y,pred))

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

          1       0.54      1.00      0.70        57
          2       0.67      0.14      0.24        14
          3       1.00      0.17      0.29         6
          4       0.00      0.00      0.00         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       0.67      0.22      0.33         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.46      0.55      0.43       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Ada boosting with Linear SVM

In [42]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import AdaBoostClassifier
param_grid = {'learning_rate':[0.0001,0.001,0.01,0.1,1]}
adaboost_svc = GridSearchCV(AdaBoostClassifier(base_estimator = linearsvc,random_state = 0, algorithm='SAMME'),
                            param_grid, cv=5,return_train_score=True)
adaboost_svc.fit(data_train_x, data_train_y)

pred = adaboost_svc.predict(data_test_x)
print(classification_report(data_test_y,pred))

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

          1       0.68      0.95      0.79        57
          2       0.60      0.43      0.50        14
          3       0.86      1.00      0.92         6
          4       1.00      0.75      0.86         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.67      0.50      0.57         4
         10       0.78      0.78      0.78         9
         15       0.00      0.00      0.00         2
         16       1.00      0.17      0.29         6

avg / total       0.64      0.70      0.64       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Ada boosting with Kernalized SVM

In [43]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import AdaBoostClassifier
param_grid = {'learning_rate':[0.0001,0.001,0.01,0.1,1]}
adaboost_ksvc = GridSearchCV(AdaBoostClassifier(base_estimator = svc,random_state = 0, algorithm='SAMME'),
                            param_grid, cv=5,return_train_score=True)
adaboost_ksvc.fit(data_train_x, data_train_y)

pred = adaboost_ksvc.predict(data_test_x)
print(classification_report(data_test_y,pred))

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

          1       0.50      1.00      0.67        57
          2       0.00      0.00      0.00        14
          3       0.00      0.00      0.00         6
          4       0.00      0.00      0.00         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       0.00      0.00      0.00         4
         10       0.00      0.00      0.00         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.25      0.50      0.34       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Ada Boosting with Decision Tree

In [44]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.ensemble import AdaBoostClassifier
param_grid = {'learning_rate':[0.0001,0.001,0.01,0.1,1]}
adaboost_dt = GridSearchCV(AdaBoostClassifier(base_estimator = dt,random_state = 0),
                            param_grid, cv=5,return_train_score=True)
adaboost_dt.fit(data_train_x, data_train_y)

pred = adaboost_dt.predict(data_test_x)
print(classification_report(data_test_y,pred))

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

          1       0.75      0.82      0.78        57
          2       0.50      0.50      0.50        14
          3       0.00      0.00      0.00         6
          4       0.50      0.75      0.60         4
          5       1.00      0.25      0.40         4
          6       0.67      0.67      0.67         6
          8       0.00      0.00      0.00         1
          9       1.00      0.50      0.67         4
         10       0.44      0.78      0.56         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.60      0.63      0.60       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Ada Boosting with Random Forest

# Gradient Boosting Classifier

In [46]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=10, n_estimators= 500)

param_grid = {'max_features':['auto', 'log2'], 'learning_rate' : [0.01,0.1], 'max_depth':[5,10,15,30,50]}

grid_search_gb = GridSearchCV(model, param_grid, cv=5)
grid_search_gb.fit(data_train_x, data_train_y)
pred = grid_search_gb.predict(data_test_x)
print(classification_report(data_test_y,pred))

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

          1       0.66      0.98      0.79        57
          2       0.55      0.43      0.48        14
          3       1.00      0.50      0.67         6
          4       1.00      0.25      0.40         4
          5       0.00      0.00      0.00         4
          6       0.00      0.00      0.00         6
          8       0.00      0.00      0.00         1
          9       1.00      0.50      0.67         4
         10       0.64      0.78      0.70         9
         15       0.00      0.00      0.00         2
         16       0.00      0.00      0.00         6

avg / total       0.57      0.66      0.59       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Comparing bagging and boosting models with previous one's
When we compare the precision and recall of earlier model with models with baaging and boosting we see that the new models accuracy has dropped. Bagging and boosting are used to lower the bias of the model, When we apply boost we introduce the risk of overfitting. Overfitting ulitmately lowers the accuracy estimated by cross-validation

# 4. PCA
We will be using principal component analysis for data reduction. We have 278 variables which is increasing the complexity of the models. PCA reduces the complexity of the model by only considering the important variables and thus reducing the overall data set.

In [49]:
from sklearn.decomposition import PCA

pca = PCA(0.95)
pca.fit(Data_X)
Data_X_PCA = pca.transform(Data_X)

data_train_x_pca, data_test_x_pca, data_train_y_pca, data_test_y_pca = train_test_split(Data_X_PCA, Data_Y, random_state = 100)

In [50]:
#Checking results of 2 scalars

from sklearn.preprocessing import MinMaxScaler

#MinMax
MinMax = MinMaxScaler(feature_range= (0,1))
data_train_x_pca = MinMax.fit_transform(data_train_x_pca)
data_test_x_pca = MinMax.transform(data_test_x_pca)

# KNN with PCA

In [51]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {'weights':['distance', 'uniform'], 'n_neighbors':range(3,100)}

grid_search_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5,return_train_score=True)
grid_search_knn.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(grid_search_knn.cv_results_)
print(grid_search_knn.best_score_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


0.5368731563421829


In [52]:
print(grid_search_knn.best_estimator_)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')


In [53]:
knn_pca = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='distance')
knn_pca.fit(data_train_x_pca, data_train_y_pca)
pred = knn_pca.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.68      0.93      0.79        72
          2       0.33      0.10      0.15        10
          3       0.00      0.00      0.00         5
          4       0.00      0.00      0.00         2
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         5
         10       0.80      0.44      0.57         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.53      0.64      0.56       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Logistic with PCA

In [54]:
from sklearn.linear_model import LogisticRegression

param_grid = {'C': range(1,100) }

grid_search_log = GridSearchCV(LogisticRegression(penalty='l1'), param_grid, cv=5,return_train_score=True)
grid_search_log.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(grid_search_log.cv_results_)
print(grid_search_log.best_estimator_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [55]:
print(grid_search_log.best_estimator_)

LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [56]:
logistic_pca = LogisticRegression(C=4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
logistic_pca.fit(data_train_x_pca, data_train_y_pca)
pred = logistic_pca.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.81      0.93      0.86        72
          2       0.57      0.40      0.47        10
          3       0.83      1.00      0.91         5
          4       0.50      1.00      0.67         2
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         5
         10       0.88      0.78      0.82         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.68      0.75      0.71       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Linear SVM with PCA

In [58]:
from sklearn.svm import LinearSVC

param_grid = {'C': [0.0001,0.001,0.01,0.1,2,3,4,5,6,7,8,9], 'max_iter':[100,1000,10000] }

grid_search_linearsvc = GridSearchCV(LinearSVC(random_state=0), param_grid, cv=5,return_train_score=True)
grid_search_linearsvc.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(grid_search_linearsvc.cv_results_)
print(grid_search_linearsvc.best_estimator_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


LinearSVC(C=2, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)


In [59]:
linearsvc = LinearSVC(C=2, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=100,
     multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
     verbose=0)
linearsvc.fit(data_train_x_pca, data_train_y_pca)
pred = linearsvc.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.81      0.92      0.86        72
          2       0.57      0.40      0.47        10
          3       0.83      1.00      0.91         5
          4       0.50      1.00      0.67         2
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         5
         10       0.75      0.67      0.71         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.68      0.73      0.70       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Kernalized SVM with PCA

In [60]:
from sklearn.svm import SVC

param_grid = {'C':[0.001, 0.01, 0.1, 0.5, 1, 10, 50, 100, 1000], 'gamma':[0.001, 0.01, 0.1, 0.5, 1, 10]}

grid_search_svc = GridSearchCV(SVC(kernel = 'rbf'), param_grid, cv=5,return_train_score=True)
grid_search_svc.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(grid_search_svc.cv_results_)
print(grid_search_svc.best_estimator_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [61]:
svc = SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
svc.fit(data_train_x_pca, data_train_y_pca)
pred = svc.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.78      0.97      0.86        72
          2       0.50      0.30      0.37        10
          3       0.80      0.80      0.80         5
          4       0.67      1.00      0.80         2
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         5
         10       0.75      0.67      0.71         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.65      0.75      0.69       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Decision tree with PCA

In [62]:
from sklearn.tree import DecisionTreeClassifier

param_grid = {'max_features':['auto', 'log2'], 'max_depth':[3,5,7,9,10,11,13,15,20,50]}

grid_search_dt = GridSearchCV(DecisionTreeClassifier(random_state = 0), param_grid, cv=5,return_train_score=True)
grid_search_dt.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(grid_search_dt.cv_results_)
print(grid_search_dt.best_estimator_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')


In [63]:
dt_pca = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')
dt_pca.fit(data_train_x_pca, data_train_y_pca)
pred = dt_pca.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.69      0.85      0.76        72
          2       0.20      0.20      0.20        10
          3       0.67      0.40      0.50         5
          4       0.00      0.00      0.00         2
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         5
         10       0.18      0.22      0.20         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.50      0.59      0.54       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Random Forest with PCA

In [64]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

#Tuning ridge on new dataset
param_grid = {"max_depth": sp_randint(3, 9),
              "max_features": sp_randint(1, 10),
              "min_samples_split": sp_randint(2, 20),
              "min_samples_leaf": sp_randint(1, 20),
              "bootstrap": [True, False]}
random_search_rf = RandomizedSearchCV(RandomForestClassifier(n_estimators=1000), param_distributions=param_grid,
                                   n_iter=30, random_state=0,n_jobs=-1, cv= 5,return_train_score=True)
random_search_rf.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(random_search_rf.cv_results_)
print(random_search_rf.best_estimator_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=5, max_features=9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [65]:
rf_pca = RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=5, max_features=9, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=2, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
rf_pca.fit(data_train_x_pca, data_train_y_pca)
pred = rf_pca.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.75      0.92      0.83        72
          2       0.50      0.50      0.50        10
          3       0.75      0.60      0.67         5
          4       0.00      0.00      0.00         2
          5       0.00      0.00      0.00         1
          6       0.00      0.00      0.00         5
         10       0.36      0.44      0.40         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.58      0.69      0.63       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Gradient boosting with PCA

In [66]:
param_grid = {"max_depth": sp_randint(5,30),
              "max_features": sp_randint(1, 35),
              "min_samples_split": sp_randint(2, 40),
              "min_samples_leaf": sp_randint(1, 30),
             'learning_rate': [0.01,0.001,0.1]}
random_search_gb = RandomizedSearchCV(GradientBoostingClassifier(n_estimators=1000), param_distributions=param_grid,
                                   n_iter=30, random_state=0,n_jobs=-1,cv= 5,return_train_score=True)
random_search_gb.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(random_search_gb.cv_results_)
print(random_search_gb.best_estimator_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=9,
              max_features=6, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=7, min_samples_split=19,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)


In [67]:
gbc_pca = GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=9,
              max_features=6, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=7, min_samples_split=19,
              min_weight_fraction_leaf=0.0, n_estimators=1000,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
gbc_pca.fit(data_train_x_pca, data_train_y_pca)
pred = gbc_pca.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.79      0.93      0.85        72
          2       0.67      0.60      0.63        10
          3       0.83      1.00      0.91         5
          4       0.50      0.50      0.50         2
          5       0.00      0.00      0.00         1
          6       0.50      0.20      0.29         5
         10       0.62      0.56      0.59         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.68      0.75      0.71       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# ADA boosting with PCA

In [68]:
param_grid = {'learning_rate' :[0.0001,0.001,0.01,0.1,1,1.2,2] ,
              "algorithm":['SAMME', 'SAMME.R']}
random_search_ada = RandomizedSearchCV(AdaBoostClassifier(n_estimators = 1000), param_distributions=param_grid,
                                   n_iter=10, random_state=0,n_jobs=-1,cv= 5,return_train_score=True)
random_search_ada.fit(data_train_x_pca, data_train_y_pca)
pd.DataFrame(random_search_ada.cv_results_)
print(random_search_ada.best_estimator_)

C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=1,
          n_estimators=1000, random_state=None)


In [69]:
ada_pca = AdaBoostClassifier(algorithm='SAMME', base_estimator=None, learning_rate=1,
          n_estimators=1000, random_state=None)
ada_pca.fit(data_train_x_pca, data_train_y_pca)
pred = ada_pca.predict(data_test_x_pca)
print(classification_report(data_test_y_pca,pred))

             precision    recall  f1-score   support

          1       0.73      0.85      0.78        72
          2       0.00      0.00      0.00        10
          3       0.67      0.40      0.50         5
          4       0.00      0.00      0.00         2
          5       0.00      0.00      0.00         1
          6       0.50      0.20      0.29         5
          8       0.00      0.00      0.00         0
          9       0.00      0.00      0.00         0
         10       0.31      0.44      0.36         9
         14       0.00      0.00      0.00         1
         15       0.00      0.00      0.00         3
         16       0.00      0.00      0.00         5

avg / total       0.54      0.60      0.56       113



C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\harsh\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


# Evaluation after PCA
The models started performing better after we applied PCA on the original data. The reason behind this is, PCA reduces the complexity of the data. It creates components based on giving importance to variables with large variance and also the components which it creates are non collinear in nature which means it takes care of collinearity in large data set. PCA also improves the overall execution time and quality of the models and it is very beneficial when we are working with huge amount of variables.
The Best model according to the precision and recall score is Kernalized SVM with PCA having accuracy of 75%.

## End